# Imports

In [1]:
import xml

# Exploring Data

In [4]:
year = ['1986', '1994', '1998']
document = xml.dom.minidom.parse(f'data/Political_election_manifestos/VP_{year[0]}.party-topicnr-content.xml')


In [19]:
chapter = document.firstChild.getElementsByTagName('chapter')[0]

In [26]:
p = chapter.getElementsByTagName('p')[0]

In [66]:
test = p.childNodes

In [68]:
for t in test:
    print(t.localName)

None
themes
None


In [111]:
test = {}
test[0] = {'text':'blabla', 'themes':['here', 'here']}

# test[1] = 1
# test[1]['text'] = 'bliblib'
# test[1]['themes'] = ['there', 'there']
print(test)

AttributeError: 'dict' object has no attribute 'add'

In [131]:
chapters = {}
for chapter in document.firstChild.getElementsByTagName('chapter'):
    party = chapter.getAttribute('party')
    # print(chapter.getAttribute('party'))
    par = {}
    for paragraph in chapter.getElementsByTagName('p'):
        id = paragraph.getAttribute('id')
        # print(paragraph.getAttribute('id'))
        
        paragraph_value = paragraph.childNodes[2].nodeValue.strip()
        # print(paragraph_value)

        paragraph_themes = [theme.getAttribute('id') for theme in paragraph.getElementsByTagName('theme')]
        # print(paragraph_themes)

        par[id] = {'p': paragraph_value, 'themes': paragraph_themes}
    chapters[party] = par

In [134]:
for key in chapters.keys():
    print(key)

nl.p.cda
nl.p.cpn
nl.p.d66
nl.p.evp
nl.p.gpv
nl.p.ppr
nl.p.psp
nl.p.pvda
nl.p.rpf
nl.p.sgp
nl.p.vvd
